In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
from photutils import CircularAperture
from photutils import aperture_photometry

In [2]:
diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
medianDir = '/media/mj1e16/PP AV-TV/medians/'
imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'

In [4]:
diffImages = os.listdir(diffDir)
diffImages = [x for x in diffImages if '.fits' in x]

medianImages = os.listdir(medianDir)
medianImages = [x for x in medianImages if '.fits' in x]

images = os.listdir(imageDir)
images = [x for x in images if '.fits' in x]

In [9]:
unmatchedObjects = ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')

In [81]:
xpos = np.array(unmatchedObjects['X_IMAGE'])
ypos = np.array(unmatchedObjects['Y_IMAGE'])
imname = np.array(unmatchedObjects['NAME'])
index = np.array(unmatchedObjects['index'])
allnames = np.unique(imname)

In [34]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [36]:
image2diff = {}
image2median = {}
for x in diffImages:
    results = findName(x)
    image2diff[results[0]] = x
    image2median[results[0]] = results[1]

In [85]:
#def measureBoth(ra,dec,names,name):
name = allnames[0]
coords = [(x,y) for x,y,z in zip(xpos,ypos,imname) if z==name]
namelist = [x for x in imname if x == name]
indexing = [x for x,y in zip(index,imname) if y==name]
apertures = CircularAperture(coords, r=2.)

locator1 = '['
locator2 = ']'
loc1 = name.find(locator1)
loc2 = name.find(locator2)
print(loc1,loc2)
shortname = name[:loc1]
number = int(name[loc1+1:loc2])
print(number)

hdu = fits.open(imageDir+shortname)
data = hdu[number].data
phot_table_diff = aperture_photometry(data, apertures)
phot_table_diff['name'] = namelist

imcounts = np.asarray(phot_table_diff['aperture_sum'])
imcounts.tolist()
hdu = fits.open(medianDir+image2median[allnames[0]])
data = hdu[0].data
phot_table_median = aperture_photometry(data, apertures)
phot_table_median['name'] = namelist

mediancounts = np.asarray(phot_table_median['aperture_sum'])
mediancounts.tolist()
magdiff = [-2.5*np.log10(x/y) for x,y in zip(imcounts,mediancounts)]

(30, 33)
10


In [86]:
magdiff

[-0.00188563682814637,
 -0.014196617478844003,
 -0.15387664851563995,
 -0.0162191958982992,
 -0.0756499120961481,
 -0.741003410233525,
 0.0001315730112397266,
 -0.14101224963263997,
 -0.0795564368950285,
 -0.023054193395557063,
 -0.02143638838292886,
 -0.06799825627979975,
 -0.08084921437982,
 -0.01491096595079783,
 -0.010916447500645788,
 -0.022033605736506288,
 -0.0784917112573032,
 -0.11329236273764884,
 -0.06182793944697662,
 -0.09458168139931314,
 -0.18656909443090622,
 -0.7658130550550348,
 -0.012729843697183548,
 -0.24108443457779977,
 -0.016978977377919726,
 -0.04623100684017765,
 -0.013402042282163157,
 -0.06278230049493959,
 -0.015486143773230314,
 -0.07111609381758158,
 -0.020642821368611904,
 -0.11127936454841565,
 -0.07650497125231494,
 -0.00795431581255603,
 -0.042622335151258774,
 -0.06435288705935525,
 -0.15263580792231815,
 -0.07114003819971736,
 -0.1413898475392896,
 -0.015727859589081645,
 -0.022634672463675776,
 -0.006745258838685782,
 -0.06260669971878331,
 -0.0575

In [80]:
phot_table_diff

id,xcenter,ycenter,aperture_sum,name
,pix,pix,,
int64,float64,float64,float64,str34
1,1031.4093,24.3685,270882.05309663335,kplr2009114174833_ffi-cal.fits[10]
2,322.944,23.0232,24867.044537262434,kplr2009114174833_ffi-cal.fits[10]
3,1094.0135,42.9983,2574.558000088245,kplr2009114174833_ffi-cal.fits[10]
4,468.0434,51.0944,34852.41238119573,kplr2009114174833_ffi-cal.fits[10]
5,145.6611,56.9963,2368.527864465521,kplr2009114174833_ffi-cal.fits[10]
6,1067.4344,1055.833,22.905996529412253,kplr2009114174833_ffi-cal.fits[10]
7,826.9632,1056.9746,77632.89468990071,kplr2009114174833_ffi-cal.fits[10]
8,229.9743,1006.0083,3253.518082577203,kplr2009114174833_ffi-cal.fits[10]


In [ ]:
positions = [(30., 30.), (40., 40.)]
apertures = CircularAperture(positions, r=3.)
phot_table = aperture_photometry(data, apertures)
#phot_table['aperture_sum'].info.format = '%.8g' 